# <center>WORKING WITH TABLE DATA WITH R</center>
<img src="../elem/caldiss_symbol_square.png" width="200">


<i><center>Kristian Gade Kjelmann</center></i>
<i><center>March 5th 2020</center><i>

## Disposition

- Indlæs og udforsk tabeldata
- Subsetting
- Summary statistics
- R til datahåndtering - dplyr/tidyverse
    - Rekodningsbasics
    - Filter og selekter
- Numeriske variable
- Tekst variable
- Kategoriske variable

In [15]:
library(haven)

ess_data <- read_dta("https://github.com/CALDISS-AAU/workshop_r-table-data/raw/master/data/ess2014_mainsub_p1.dta")

In [16]:
head(ess_data)

idno,ppltrst,polintr,vote,lrscale,happy,health,cgtsday,cgtsmke,alcfreq,brncntr,height,weight,gndr,yrbrn,edlvddk,marsts,polpartvt
<dbl>,<dbl+lbl>,<dbl+lbl>,<dbl+lbl>,<dbl+lbl>,<dbl+lbl>,<dbl+lbl>,<dbl>,<dbl+lbl>,<dbl+lbl>,<dbl+lbl>,<dbl>,<dbl>,<dbl+lbl>,<dbl>,<dbl+lbl>,<dbl+lbl>,<chr>
921018,7,3,3,5,10,1,10,2,4,1,178,64,1,1990,2,6,[NA] Not applicable
921026,9,2,1,5,9,1,NA,5,2,1,172,64,2,1948,8,5,[1] Socialdemokraterne - the Danish social democrats
921034,9,2,1,8,9,2,NA,3,1,1,176,87,1,1957,7,7,"[7] Venstre, Danmarks Liberale Parti - Venstre"
921181,10,2,1,6,10,3,NA,3,3,1,194,102,1,1956,6,7,[2] Det Radikale Venstre - Danish Social-Liberal Party
921204,10,3,1,8,9,2,NA,3,3,2,157,48,2,1941,7,7,[NA] Don't know
921262,9,3,1,8,9,1,NA,5,4,1,180,93,1,1987,8,6,"[7] Venstre, Danmarks Liberale Parti - Venstre"


In [17]:
summary(ess_data)

      idno           ppltrst          polintr           vote     
 Min.   :921018   Min.   : 1.000   Min.   :1.000   Min.   :1.00  
 1st Qu.:928301   1st Qu.: 7.000   1st Qu.:2.000   1st Qu.:1.00  
 Median :935845   Median : 8.000   Median :2.000   Median :1.00  
 Mean   :935664   Mean   : 7.915   Mean   :2.136   Mean   :1.25  
 3rd Qu.:942857   3rd Qu.: 9.000   3rd Qu.:3.000   3rd Qu.:1.00  
 Max.   :950516   Max.   :12.000   Max.   :5.000   Max.   :4.00  
                                                                 
    lrscale           happy            health        cgtsday      
 Min.   : 1.000   Min.   : 2.000   Min.   :1.00   Min.   :  0.00  
 1st Qu.: 5.000   1st Qu.: 9.000   1st Qu.:1.00   1st Qu.:  5.00  
 Median : 7.000   Median : 9.000   Median :2.00   Median : 10.00  
 Mean   : 6.742   Mean   : 9.144   Mean   :1.94   Mean   : 11.51  
 3rd Qu.: 9.000   3rd Qu.:10.000   3rd Qu.:3.00   3rd Qu.: 17.00  
 Max.   :14.000   Max.   :14.000   Max.   :5.00   Max.   :102.00  
   

In [21]:
colnames(ess_data)

[1] "idno"      "ppltrst"   "polintr"   "vote"      "lrscale"   "happy"    
 [7] "health"    "cgtsday"   "cgtsmke"   "alcfreq"   "brncntr"   "height"   
[13] "weight"    "gndr"      "yrbrn"     "edlvddk"   "marsts"    "polpartvt"

In [22]:
head(ess_data$gndr)

<Labelled double>
[1] 1 2 1 1 2 1

Labels:
 value     label
     1      Male
     2    Female
     3 No answer

In [23]:
ess_data[c(1:5), c("gndr", "alcfreq")]

gndr,alcfreq
<dbl+lbl>,<dbl+lbl>
1,4
2,2
1,1
1,3
2,3
